In [2]:
import util
# Standard Library Imports

# TabPFN and Extensions

try:
    from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import (
        AutoTabPFNClassifier,
    )

    from tabpfn import TabPFNClassifier, TabPFNRegressor
except ImportError:
    raise ImportError(
        "Warning: Could not import TabPFN / TabPFN extensions. Please run installation above and restart the session afterwards (Runtime > Restart Session)."
    )

# Data Science & Visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

# Other ML Models
from catboost import CatBoostClassifier, CatBoostRegressor

# Notebook UI/Display
from IPython.display import Markdown, display
from rich.console import Console
from rich.panel import Panel
from rich.prompt import Prompt
from rich.rule import Rule
from sklearn.compose import make_column_selector, make_column_transformer

# Scikit-Learn: Data & Preprocessing
from sklearn.datasets import fetch_openml, load_breast_cancer

# Scikit-Learn: Models
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import (
    KFold,
    StratifiedKFold,
    cross_val_score,
    train_test_split,
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from xgboost import XGBClassifier, XGBRegressor

# This transformer will be used to handle categorical features for the baseline models
column_transformer = make_column_transformer(
    (
        OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
        make_column_selector(dtype_include=["object", "category"]),
    ),
    remainder="passthrough",
)

/Users/loganyamamoto/Desktop/class/CSCI/566/project/StockPredictor/.venv/lib/python3.11/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
console = Console()
console.print("Attempting client backend setup...")
console.print("Importing TabPFN client library...")
from tabpfn_client import TabPFNClassifier, TabPFNRegressor, init

init()
console.print("[bold green]✅ TabPFN (client) initialized.[/bold green]")

Attempting client backend setup...

Importing TabPFN client library...

Found existing access token, reusing it for authentication.

✅ TabPFN (client) initialized.

In [4]:
stocks = [
# Communication Services
"GOOGL", "GOOG", "T", "CHTR", "CMCSA", "EA", "FOXA", "FOX", "IPG", "LYV", "MTCH", "META", "NFLX", "NWSA", "NWS", "OMC", "PSKY", "TMUS", "TTWO", "TKO", "TTD", "VZ", "DIS", "WBD",

# consumer discretionary
"ABNB", "AMZN", "APTV", "AZO", "BBY", "BKNG", "CZR", "KMX", "CCL", "CMG", "DRI", "DECK", "DPZ", "DASH", "DHI", "EBAY", "EXPE", "F", "GRMN", "GM", "GPC", "HAS", "HLT", "HD", "LVS", "LEN", "LKQ", "LOW", "LULU", "MAR", "MCD", "MGM", "MHK", "NKE", "NCLH", "NVR", "ORLY", "POOL", "PHM", "RL", "ROST", "RCL", "SBUX", "TPR", "TSLA", "TJX", "TSCO", "ULTA", "WSM", "WYNN", "YUM",

# Consumer Staples
"MO", "ADM", "BF.B", "BG", "CPB", "CHD", "CLX", "KO", "CL", "CAG", "STZ", "COST", "DG", "DLTR", "EL", "GIS", "HSY", "HRL", "K", "KVUE", "KDP", "KMB", "KHC", "KR", "LW", "MKC", "TAP", "MDLZ", "MNST", "PEP", "PM", "PG", "SJM", "SYY", "TGT", "TSN", "WBA", "WMT",

# Energy
"APA", "BKR", "CVX", "COP", "CTRA", "DVN", "FANG", "EOG", "EQT", "EXE", "XOM", "HAL", "KMI", "MPC", "OXY", "OKE", "PSX", "SLB", "TRGP", "TPL", "VLO", "WMB",

# Financials
"AFL", "ALL", "AXP", "AIG", "AMP", "AON", "APO", "ACGL", "AJG", "AIZ", "BAC", "BRK.B", "BLK", "BX", "XYZ", "BK", "BRO", "COF", "CBOE", "SCHW", "CB", "CINF", "C", "CFG", "CME", "COIN", "CPAY", "ERIE", "EG", "FDS", "FIS", "FITB", "FI", "BEN", "GPN", "GL", "GS", "HIG", "HBAN", "ICE", "IVZ", "JKHY", "JPM", "KEY", "KKR", "L", "MTB", "MKTX", "MMC", "MA", "MET", "MCO", "MS", "MSCI", "NDAQ", "NTRS", "PYPL", "PNC", "PFG", "PGR", "PRU", "RJF", "RF", "SPGI", "STT", "SYF", "TROW", "TRV", "TFC", "USB", "V", "WRB", "WFC", "WTW",

# Healthcare
"ABT", "ABBV", "A", "ALGN", "AMGN", "BAX", "BDX", "TECH", "BIIB", "BSX", "BMY", "CAH", "COR", "CNC", "CRL", "CI", "COO", "CVS", "DHR", "DVA", "DXCM", "EW", "ELV", "GEHC", "GILD", "HCA", "HSIC", "HOLX", "HUM", "IDXX", "INCY", "PODD", "ISRG", "IQV", "JNJ", "LH", "LLY", "MCK", "MDT", "MRK", "MTD", "MRNA", "MOH", "PFE", "DGX", "REGN", "RMD", "RVTY", "SOLV", "STE", "SYK", "TMO", "UNH", "UHS", "VRTX", "VTRS", "WAT", "WST", "ZBH", "ZTS",

# Industrials
"MMM", "AOS", "ALLE", "AME", "ADP", "AXON", "BA", "BR", "BLDR", "CHRW", "CARR", "CAT", "CTAS", "CPRT", "CSX", "CMI", "DAY", "DE", "DAL", "DOV", "ETN", "EMR", "EFX", "EXPD", "FAST", "FDX", "FTV", "GE", "GEV", "GNRC", "GD", "HON", "HWM", "HUBB", "HII", "IEX", "ITW", "IR", "JBHT", "J", "JCI", "LHX", "LDOS", "LII", "LMT", "MAS", "NDSN", "NSC", "NOC", "ODFL", "OTIS", "PCAR", "PH", "PAYX", "PAYC", "PNR", "PWR", "RTX", "RSG", "ROK", "ROL", "SNA", "LUV", "SWK", "TXT", "TT", "TDG", "UBER", "UNP", "UAL", "UPS", "URI", "VLTO", "VRSK", "GWW", "WAB", "WM", "XYL",

# Information Technology
"ACN", "ADBE", "AMD", "AKAM", "APH", "ADI", "AAPL", "AMAT", "ANET", "ADSK", "AVGO", "CDNS", "CDW", "CSCO", "CTSH", "GLW", "CRWD", "DDOG", "DELL", "ENPH", "EPAM", "FFIV", "FICO", "FSLR", "FTNT", "IT", "GEN", "GDDY", "HPE", "HPQ", "IBM", "INTC", "INTU", "JBL", "KEYS", "KLAC", "LRCX", "MCHP", "MU", "MSFT", "MPWR", "MSI", "NTAP", "NVDA", "NXPI", "ON", "ORCL", "PLTR", "PANW", "PTC", "QCOM", "ROP", "CRM", "STX", "NOW", "SWKS", "SMCI", "SNPS", "TEL", "TDY", "TER", "TXN", "TRMB", "TYL", "VRSN", "WDC", "WDAY", "ZBRA",

# Materials
"APD", "ALB", "AMCR", "AVY", "BALL", "CF", "CTVA", "DOW", "DD", "EMN", "ECL", "FCX", "IFF", "IP", "LIN", "LYB", "MLM", "MOS", "NEM", "NUE", "PKG", "PPG", "SHW", "SW", "STLD", "VMC",

# Real Estate
"ARE", "AMT", "AVB", "BXP", "CPT", "CBRE", "CSGP", "CCI", "DLR", "EQIX", "EQR", "ESS", "EXR", "FRT", "DOC", "HST", "INVH", "IRM", "KIM", "MAA", "PLD", "PSA", "O", "REG", "SBAC", "SPG", "UDR", "VTR", "VICI", "WELL", "WY",

# Utilities
"AES", "LNT", "AEE", "AEP", "AWK", "ATO", "CNP", "CMS", "ED", "CEG", "D", "DTE", "DUK", "EIX", "ETR", "EVRG", "ES", "EXC", "FE", "NEE", "NI", "NRG", "PCG", "PNW", "PPL", "PEG", "SRE", "SO", "VST", "WEC", "XEL"
]


start = "1990-01-01"
#end = "1991-01-01"
end = "2015-12-31"
time_args = [start,end]
prediction_type = "classification"
use_nlp = True
nlp_method = "aggregated"

In [5]:
# Step 1: Check cache first with full stock list
input_data = util.load_data_from_cache(stocks, time_args, data_dir="data", prediction_type=prediction_type, use_nlp=use_nlp, nlp_method=nlp_method)

if input_data is None:
    # Step 2: Load saved problematic stocks for this time period
    problematic_stocks_saved = util._load_problematic_stocks(time_args, data_dir="data")
    
    # Step 3: Remove problematic stocks from input set
    if problematic_stocks_saved:
        valid_stocks = [stock for stock in stocks if stock not in problematic_stocks_saved]
        print(f"[data] Loaded {len(problematic_stocks_saved)} previously identified problematic stocks for this time period")
        print(f"[data] Filtered input: {len(stocks)} -> {len(valid_stocks)} stocks")
    else:
        valid_stocks = stocks
    
    if len(valid_stocks) == 0:
        raise RuntimeError("No valid stocks found after filtering problematic stocks.")
    
    # Step 4: Check cache with filtered stocks
    input_data = util.load_data_from_cache(valid_stocks, time_args, data_dir="data", prediction_type=prediction_type, use_nlp=use_nlp, nlp_method=nlp_method)
    
    if input_data is None:
        # Step 5: Download data (will save problematic stocks for future runs)
        print("[data] Cache not found, downloading data...")
        open_close, failed_stocks_dict = util.handle_yfinance_errors(valid_stocks, time_args, max_retries=1)
        
        if open_close is None:
            raise RuntimeError("ERROR: Failed to download any stock data. Cannot proceed.")
        
        # Calculate problematic stocks from this download (new problematic stocks found in valid_stocks)
        new_problematic = [stock for stock in valid_stocks if stock not in open_close["Open"].columns]
        # Combine with previously known problematic stocks
        all_problematic = list(problematic_stocks_saved) + new_problematic if problematic_stocks_saved else new_problematic
        
        # Step 6: Process and save data (get_data will save problematic stocks)
        print("[data] Processing downloaded data and saving to cache...")
        input_data = util.get_data(valid_stocks, time_args, data_dir="data", prediction_type=prediction_type, open_close_data=open_close, problematic_stocks=all_problematic if all_problematic else None, use_nlp=use_nlp, nlp_method=nlp_method)
        if isinstance(input_data, int):
            raise RuntimeError("Error getting data from util.get_data()")
    else:
        print("[data] Found cache for filtered stocks")

Loading training dataset (.npz):   0%|          | 0/3 [00:00<?, ?it/s]

Loading metrics dataset (.npz): 100%|██████████| 3/3 [00:00<00:00, 408.39it/s]


In [6]:
print(len(input_data))

12


In [7]:
if len(input_data) == 10:
    X_train, X_val, X_test, Y_train, Y_val, Y_test, D_train, D_val, D_test, Rev_test = input_data
    Returns_test = None  # Old format doesn't have Returns
    Sp500_test = None  # Old format doesn't have S&P 500
elif len(input_data) == 11:
    X_train, X_val, X_test, Y_train, Y_val, Y_test, D_train, D_val, D_test, Rev_test, Sp500_test = input_data
    Returns_test = None  # Format doesn't have Returns
elif len(input_data) == 12:
    X_train, X_val, X_test, Y_train, Y_val, Y_test, D_train, D_val, D_test, Rev_test, Returns_test, Sp500_test = input_data
else:
    raise ValueError(f"Unexpected number of elements in input_data: {len(input_data)}. Expected 10, 11, or 12 elements.")

In [8]:
print(X_train.shape)

torch.Size([1445936, 31, 13])


In [9]:
X_train = X_train.flatten(start_dim=1)

In [10]:
print(X_train.shape)

torch.Size([1445936, 403])


In [ ]:
import numpy as np
import pickle
from tabpfn import save_fitted_tabpfn_model

# TabPFN has a limit of 50,000 rows
CHUNK_SIZE = 50000

# Convert to numpy if it's a torch tensor
if isinstance(X_train, torch.Tensor):
    X_train_np = X_train.numpy()
    Y_train_np = Y_train.numpy()
else:
    X_train_np = X_train
    Y_train_np = Y_train

# Flatten Y_train to 1D array (fixes the warning)
if Y_train_np.ndim > 1:
    Y_train_np = Y_train_np.ravel()

# Get total number of samples
total_samples = X_train_np.shape[0]
num_chunks = int(np.ceil(total_samples / CHUNK_SIZE))

print(f"Total samples: {total_samples}")
print(f"Chunk size: {CHUNK_SIZE}")
print(f"Number of chunks: {num_chunks}")

# IMPORTANT: TabPFN doesn't support incremental fitting!
# Each fit() call replaces the previous state, so we can only use ONE chunk.
# Option 1: Use the last chunk (most recent data)
# Option 2: Use a random sample
# Option 3: Use the first chunk



X_chunk = X_train_np[:1000]
Y_chunk = Y_train_np[:1000]


# Initialize the model
model = TabPFNClassifier(random_state=42)

# Fit the model on the selected chunk
print(f"\nFitting model on {len(X_chunk)} samples...")
model = model.fit(X_chunk, Y_chunk)
print("✅ Model fitted successfully!")

# Check if model has executor_ attribute (required for save_fitted_tabpfn_model)
has_executor = hasattr(model, 'executor_')
print(f"\nModel has 'executor_' attribute: {has_executor}")

# Try to save using save_fitted_tabpfn_model first
model_path = "my_fitted_tabpfn_model.pkl"
try:
    if has_executor:
        save_fitted_tabpfn_model(model, model_path)
        print(f"💾 Saved fitted model using save_fitted_tabpfn_model to '{model_path}'")
    else:
        raise AttributeError("Model doesn't have executor_ attribute")
except Exception as e:
    print(f"⚠️  save_fitted_tabpfn_model failed: {e}")
    print("Trying alternative: saving with pickle...")
    # Fallback to pickle
    try:
        with open(model_path, 'wb') as f:
            pickle.dump(model, f)
        print(f"💾 Saved fitted model using pickle to '{model_path}'")
    except Exception as e2:
        print(f"❌ Pickle save also failed: {e2}")
        raise


Total samples: 1445936
Chunk size: 50000
Number of chunks: 29

Fitting model on 1000 samples...
✅ Model fitted successfully!

Model has 'executor_' attribute: False
⚠️  save_fitted_tabpfn_model failed: Model doesn't have executor_ attribute
Trying alternative: saving with pickle...
💾 Saved fitted model using pickle to 'my_fitted_tabpfn_model.pkl'


In [ ]:
import numpy as np
import pickle
from tabpfn import save_fitted_tabpfn_model

# TabPFN has a limit of 50,000 rows
CHUNK_SIZE = 50000

# Convert to numpy if it's a torch tensor
if isinstance(X_train, torch.Tensor):
    X_train_np = X_train.numpy()
    Y_train_np = Y_train.numpy()
else:
    X_train_np = X_train
    Y_train_np = Y_train

# Flatten Y_train to 1D array (fixes the warning)
if Y_train_np.ndim > 1:
    Y_train_np = Y_train_np.ravel()

# Get total number of samples
total_samples = X_train_np.shape[0]
num_chunks = int(np.ceil(total_samples / CHUNK_SIZE))

print(f"Total samples: {total_samples}")
print(f"Chunk size: {CHUNK_SIZE}")
print(f"Number of chunks: {num_chunks}")

# IMPORTANT: TabPFN doesn't support incremental fitting!
# Each fit() call replaces the previous state, so we can only use ONE chunk.
# Option 1: Use the last chunk (most recent data)
# Option 2: Use a random sample
# Option 3: Use the first chunk

# Using the last chunk (most recent data)
use_last_chunk = True
if use_last_chunk:
    start_idx = max(0, total_samples - CHUNK_SIZE)
    end_idx = total_samples
    X_chunk = X_train_np[start_idx:end_idx]
    Y_chunk = Y_train_np[start_idx:end_idx]
    print(f"\nUsing last chunk: rows {start_idx} to {end_idx-1} ({len(X_chunk)} samples)")
else:
    # Use first chunk
    X_chunk = X_train_np[0:min(CHUNK_SIZE, total_samples)]
    Y_chunk = Y_train_np[0:min(CHUNK_SIZE, total_samples)]
    print(f"\nUsing first chunk: rows 0 to {len(X_chunk)-1} ({len(X_chunk)} samples)")

# Initialize the model
model = TabPFNClassifier(random_state=42)

# Fit the model on the selected chunk
print(f"\nFitting model on {len(X_chunk)} samples...")
model.fit(X_chunk, Y_chunk)
print("✅ Model fitted successfully!")

# Check if model has executor_ attribute (required for save_fitted_tabpfn_model)
has_executor = hasattr(model, 'executor_')
print(f"\nModel has 'executor_' attribute: {has_executor}")

# Try to save using save_fitted_tabpfn_model first
model_path = "my_fitted_tabpfn_model.pkl"
try:
    if has_executor:
        save_fitted_tabpfn_model(model, model_path)
        print(f"💾 Saved fitted model using save_fitted_tabpfn_model to '{model_path}'")
    else:
        raise AttributeError("Model doesn't have executor_ attribute")
except Exception as e:
    print(f"⚠️  save_fitted_tabpfn_model failed: {e}")
    print("Trying alternative: saving with pickle...")
    # Fallback to pickle
    try:
        with open(model_path, 'wb') as f:
            pickle.dump(model, f)
        print(f"💾 Saved fitted model using pickle to '{model_path}'")
    except Exception as e2:
        print(f"❌ Pickle save also failed: {e2}")
        raise


Total samples: 1445936
Chunk size: 50000
Number of chunks: 29

Processing chunk 1/29 (rows 0 to 49999)...


/Users/loganyamamoto/Desktop/class/CSCI/566/project/StockPredictor/.venv/lib/python3.11/site-packages/tabpfn_client/estimator.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


✅ Completed chunk 1/29

Processing chunk 2/29 (rows 50000 to 99999)...


/Users/loganyamamoto/Desktop/class/CSCI/566/project/StockPredictor/.venv/lib/python3.11/site-packages/tabpfn_client/estimator.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


✅ Completed chunk 2/29

Processing chunk 3/29 (rows 100000 to 149999)...


...

/Users/loganyamamoto/Desktop/class/CSCI/566/project/StockPredictor/.venv/lib/python3.11/site-packages/tabpfn_client/estimator.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


✅ Completed chunk 28/29

Processing chunk 29/29 (rows 1400000 to 1445935)...


/Users/loganyamamoto/Desktop/class/CSCI/566/project/StockPredictor/.venv/lib/python3.11/site-packages/tabpfn_client/estimator.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


✅ Completed chunk 29/29

🎉 Finished fitting model on all 29 chunks!


RuntimeError: Estimator must be fitted before saving.

In [1]:
# Save the final fitted model
save_fitted_tabpfn_model(model, "my_fitted_tabpfn_model.pkl")
print("💾 Saved fitted model to 'my_fitted_tabpfn_model.pkl'")

NameError: name 'save_fitted_tabpfn_model' is not defined

In [ ]:
# Later, load the fitted model
import pickle
from tabpfn import load_fitted_tabpfn_model

model_path = "my_fitted_tabpfn_model.pkl"

try:
    # Try loading with load_fitted_tabpfn_model first
    loaded_model = load_fitted_tabpfn_model(model_path)
    print("✅ Loaded model using load_fitted_tabpfn_model")
except Exception as e:
    print(f"⚠️  load_fitted_tabpfn_model failed: {e}")
    print("Trying alternative: loading with pickle...")
    # Fallback to pickle
    try:
        with open(model_path, 'rb') as f:
            loaded_model = pickle.load(f)
        print("✅ Loaded model using pickle")
    except Exception as e2:
        print(f"❌ Pickle load also failed: {e2}")
        raise

ValueError: The number of rows cannot be more than 50000.

In [ ]:
# Test loading the model with pickle and make a prediction
import pickle
import numpy as np

model_path = "my_fitted_tabpfn_model.pkl"

# Load model using pickle
print("Loading model with pickle...")
try:
    with open(model_path, 'rb') as f:
        loaded_model = pickle.load(f)
    print("✅ Successfully loaded model with pickle!")
    
    # Verify the model is usable by checking its type
    print(f"Model type: {type(loaded_model)}")
    print(f"Model has 'predict' method: {hasattr(loaded_model, 'predict')}")
    
    # Optional: Test prediction on a small sample
    # Note: This requires X_test or X_val to be available
    if 'X_test' in globals() or 'X_val' in globals():
        test_X = X_val[:10] if 'X_val' in globals() else X_test[:10]
        if isinstance(test_X, torch.Tensor):
            test_X = test_X.numpy()
        if test_X.ndim > 2:
            test_X = test_X.reshape(test_X.shape[0], -1)
        
        print(f"\nTesting prediction on {len(test_X)} samples...")
        predictions = loaded_model.predict(test_X)
        print(f"✅ Predictions shape: {predictions.shape}")
        print(f"Sample predictions: {predictions[:5]}")
    
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    import traceback
    traceback.print_exc()
